### Create a GIF of Landsat images 

inspired from https://stackstac.readthedocs.io/en/latest/examples/gif.html


In [ ]:
from dask.distributed import LocalCluster,Client
import dask.utils
import pystac_client
import stackstac
import dask.array as da
from geogif import gif, dgif
import geopandas as gpd
import planetary_computer as pc
import hvplot


from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry

In [ ]:
# port forwarding : ssh -L 8000:localhost:8787 protect
# ou
# ssh -L 9998:129.88.193.194:22 ssh-ige
# ssh -L 9999:localhost:8787 -N -p 9998 localhost
# opne localhost:9999

cluster = LocalCluster(n_workers=20,
                       threads_per_worker=1,
                       dashboard_address=8787,
                       memory_limit='2GB')
   
#cluster = LocalCluster()   
client = Client(cluster)

display(client)

In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = client.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)

In [ ]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [ ]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [ ]:
# see : https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/landsat-8-c2-l2/landsat-8-c2-l2-example.ipynb

bbox =aoi.unary_union.bounds

LandsatSTAC = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

#for collection in LandsatSTAC.get_collections():
    #print(collection)

search = (
    LandsatSTAC
    .search(
        bbox=bbox,
        query =  {"eo:cloud_cover":{"lt":15}},
        datetime = "2018-01-01/2018-06-30", 
        collections = ["landsat-8-c2-l2"]
               
    )
)

In [ ]:
items = pc.sign(search)
print(str(len(items))+ ' scenes found')

Turn STAC items into xarray as a temporal stack, using stackstac.


In [ ]:
stack = stackstac.stack(items, chunksize=(6, 19, 405, 424),bounds_latlon=bbox, epsg = 32620, resolution=200)
stack

In [ ]:
# use common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
stack.band

In [ ]:
#See how much input data there is for just RGB. This is the amount of data we’ll end up processing
stack.sel(band=["red", "green", "blue"])

In [ ]:
# Make a bitmask---when we bitwise-and it with the data, it leaves just the 4 bits we care about
mask_bitfields = [1, 2, 3, 4]  # dilated cloud, cirrus, cloud, cloud shadow
bitmask = 0
for field in mask_bitfields:
    bitmask |= 1 << field

bin(bitmask)

In [ ]:
qa = stack.sel(band="QA_PIXEL").astype("uint16")
bad = qa & bitmask  # just look at those 4 bits

good = stack.where(bad == 0)  # mask pixels where any one of those bits are set

In [ ]:
# What's the typical interval between scenes?
#good.time.diff("time").dt.days.plot.hist();

In [ ]:
# keep rgb bands + Make annual median composites (`Q` means 2 quarters)
composites = good.sel(band=["red", "green", "blue"]).resample(time="M").median("time")
composites#.chunk(chunks=(2,3,405,424))


In [ ]:
%%time
singleimage= composites.sel(time='2018-04-30T00:00:00.000000000').compute()

In [ ]:
singleimage.plot.imshow(rgb="band", robust=True)

In [ ]:
cleaned = composites.ffill("time")
cleaned.chunk((1,3,405,424))

In [ ]:
ts = cleaned.persist()
ts_local = ts.compute()


In [ ]:
ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [ ]:
%time
#gif_img = dgif(rgb).compute()

In [ ]:
gif_data = dgif(ts,fps=8, bytes=True).compute()
with open("petermann_planetary_2016_2021.gif", "wb") as f:
        f.write(gif_data)
